In [ ]:
# !pip install ipywidgets

In [16]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import json

# This function is to output the game info as a dataframe, so the viewers could get a big picture of the chosen game
# The input (game id) is given by the interactive tools entered by the user
# SoG: shotsOnGoal, SOGoals: shootout goals, SOAttemps: shootout attempts
def get_game_output(game_id):
    game_id = str(game_id)
    file_path = "nhl_data/nhl_play_by_play_"+game_id[:4]+'_'+game_id+".json"
    df = pd.read_json(file_path)

    away_team = df["liveData"]["boxscore"]["teams"]["away"]["team"]["abbreviation"]
    away_goals = df["liveData"]["boxscore"]["teams"]["away"]["teamStats"]["teamSkaterStats"]["goals"]
    away_SoG = df["liveData"]["linescore"]["teams"]["away"]["shotsOnGoal"]
    away_SOGoals = df["liveData"]["linescore"]["shootoutInfo"]["away"]["scores"]
    away_SOAttemps = df["liveData"]["linescore"]["shootoutInfo"]["away"]["attempts"]
    home_team = df["liveData"]["boxscore"]["teams"]["home"]["team"]["abbreviation"]
    home_goals = df["liveData"]["boxscore"]["teams"]["home"]["teamStats"]["teamSkaterStats"]["goals"]
    home_SoG = df["liveData"]["linescore"]["teams"]["home"]["shotsOnGoal"]
    home_SOGoals = df["liveData"]["linescore"]["shootoutInfo"]["home"]["scores"]
    home_SOAttemps = df["liveData"]["linescore"]["shootoutInfo"]["home"]["attempts"]
    df = pd.DataFrame({'Home': [home_team, home_goals, home_SoG, home_SOGoals, home_SOAttemps],'Away': [away_team, away_goals, away_SoG, away_SOGoals, away_SOAttemps]})
    df.index = ['Teams', 'Goals', 'SoG', 'SO Goals', 'SO Attempts']
    
    return df

# This function is to output the event info as a dictionary, so the viewers could get more detailed information about the chosen event
# The input (game id and event id) is given by the interactive tools entered by the user
def get_event_output(game_id, event_id):
    game_id = str(game_id)
    file_path = "nhl_data/nhl_play_by_play_"+game_id[:4]+'_'+game_id+".json"
    with open(file_path, 'r') as f:
      data = json.load(f)
    one_event_data = data["liveData"]["plays"]["allPlays"][event_id]
    del one_event_data["about"]["goals"]
    return one_event_data

# This function is to plot where the event occurred on a rink figure, with info of the event description and time and periods
# The input (game id and event id) is given by the interactive tools entered by the user
def plot_ice_rink_with_events(game_id, event_id):
    # Load the local ice rink image
    rink_image_path = 'nhl_data/nhl_rink.png'
    img = mpimg.imread(rink_image_path)
    # plt.axis('off')  # Hide axis numbers and ticks
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.imshow(img, extent=[-100, 100, -42.5, 42.5])
    event_coordinates = get_event_output(game_id, event_id)["coordinates"]
    if event_coordinates != {}:
        x, y = event_coordinates["x"], event_coordinates["y"]
        plt.plot(x, y, 'bo')
        plt.xlabel('feet')  
        plt.ylabel('feet')
        plt.title(df["liveData"]["plays"]["allPlays"][event_id]["result"]["description"] +' \n'+ str(df["liveData"]["plays"]["allPlays"][event_id]["about"]["periodTime"])+" period "+str(df["liveData"]["plays"]["allPlays"][event_id]["about"]["period"]))
    
    plt.show()
    

# Function to update the game based on the selected game ID
def update_game(game_id, event_id):
    print(f"Loading game {game_id} with event {event_id}...")
    
    # Simulate getting event data for the game (replace with actual data)
    print(get_game_output(game_id))
    plot_ice_rink_with_events(game_id, event_id)
    print(get_event_output(game_id, event_id))

# Widget to select the game ID
game_id_text = widgets.IntText(
    value=0,
    description='Game ID:',
    disabled=False
)
# Widget to select the event ID
event_id_text = widgets.IntText(
    value=0,
    description='Event ID:',
    disabled=False
)

# Button to load the selected game
load_button = widgets.Button(
    description='Load Game',
    button_style='success',
    tooltip='Load the selected game',
)

# Use input (game id and event id) given by the interactive tools entered by the user to load the game
def load_game(button):
    game_id = game_id_text.value
    event_id = event_id_text.value
    update_game(game_id, event_id)

load_button.on_click(load_game)

# Display the widgets
display(game_id_text, event_id_text, load_button)

# Here are some example that you can try to use the interactive tool
# game_id: 2016020001
# event_id: 8

IntText(value=0, description='Game ID:')

IntText(value=0, description='Event ID:')

Button(button_style='success', description='Load Game', style=ButtonStyle(), tooltip='Load the selected game')